# Analyser un document XML

## Structurer des données

Le langage XML, grâce à sa forte rigidité, permet d’organiser des informations à la manière d’une base de données relationnelle.

Si dans le cas des formats CSV et dérivés la structuration est assurée par une combinaison de retours à la ligne et de séparateurs de champs (une ligne par enregistrement, un champ par item), elle repose pour XML sur l’utilisation de balises qui décrivent les données.

Extrait du fichier *constellations.xml* utilisé comme exemple pour les manipulations :
```xml
<constellation origin="Lacaille">
    <name fr="La Carène">Carina</name>
    <areas>
        <area unit="deg2">494.184</area>
        <area unit="percent">1.20</area>
    </areas>
</constellation>
```

## Le module `xml.etree.ElementTree`

### Charger un fichier

Ce module permet de charger en mémoire un fichier au format XML :

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('./data/constellations.xml')
root = tree.getroot()

### Les objets de type `Element`

L’objet `root` est de type `Element`. Il dispose de propriétés `tag` et `attrib` qui donnent accès aux nom de la balise ainsi qu’à un dictionnaire de ses attributs :

In [ ]:
print(type(root), root.tag, root.attrib)

 Un itérateur permet d’accéder aux enfants d’un objet de classe `Element` :

In [ ]:
for child in root:
    print(child.tag, child.attrib)

## Parcourir une arborescence

Un fichier XML est composé de balises imbriquées les unes dans les autres. Cette structure forme une *arborescence*. À partir du moment où l’on cible un élément particulier, il devient possible de parcourir toute son arborescence récursivement.

### Accéder à un élément particulier

L’accès atomique à un élément est possible, quoique malaisé :

In [ ]:
print(root[2].tag, root[2].attrib)

La méthode `.find()` renvoie le premier enfant trouvé d’un élément particulier :

In [ ]:
print(root.find('constellation').tag, root.find('constellation').attrib)

### Accéder au contenu

Le contenu textuel d’un objet de type `Element` est accessible depuis une propriété `text` :

In [ ]:
root.find('constellation').find('name').text

Le contenu d’un attribut peut quant à lui être renvoyé par la méthode `.get()` :

In [ ]:
root.find('constellation').get('origin')

Ces méthodes sont évidemment plus utiles si elles sont mobilisées par un itérateur :

In [ ]:
for c in root.iter('constellation'):
    name = c.find('name')
    print(f"{name.text} ({name.get('fr')})")

### Déplier une arborescence

Quand l’itérateur d’un objet `Element` donne accès à ses enfants directs, la méthode `.iter()` renvoie, elle, toute l’arborescence :

In [ ]:
for c in root.iter():
    print(c.tag, c.attrib)

C’est souvent plus intéressant d’envoyer le nom d’un élément à la méthode `.iter()` :

In [ ]:
for name in root.iter('name'):
    print(name.text)

La méthode `.findall()` limite les réponses aux enfants directs de l’élément courant :

In [ ]:
for c in root.findall('constellation'):
    name = c.find('name').text
    origin = c.get('origin')
    print(name, origin)

## Stratégies de recherche d’informations

Afin d’illustrer les différentes stratégies possibles pour obtenir une information, nous prendrons pour exemple la taille observable de la constellation du Triangle en degrés carrés.

### Effectuer des comparaisons

La première stratégie est de profiter des structures conditionnelles pour interroger un élément particulier :

In [ ]:
# for each constellation
for c in root.findall('constellation'):

    # get the name
    name = c.find('name')

    # is it 'Triangulum'?
    if name.text == 'Triangulum':
        # get the areas
        areas = c.find('areas')

        # print when area expressed in square degrees is found
        for area in areas:
            if area.get('unit') == 'deg2':
                print(f"{name.text} : {area.text} deg2")

### Créer un dictionnaire

Comme la lecture d’un fichier XML est séquentielle, aucun index performant n’est créé lors du chargement en mémoire de ses données. Une phase préliminaire de sélection des données essentielles dans une structure Python plus facilement manipulable peut se révéler judicieuse.

In [ ]:
constellations = dict()

for c in root.findall('constellation'):
    name = c.find('name').text
    areas = c.find('areas')
    for area in areas:
        if area.get('unit') == 'deg2':
            deg2 = area.text
    constellations[name] = deg2

print(constellations['Triangulum'])

### Expressions XPath

Une autre stratégie consiste à tirer profit du support des expressions XPath :

In [ ]:
triangulum = root.find(".//constellation[name='Triangulum']")
triangulum_area = triangulum.find(".//area[@unit='deg2']")
triangulum_area.text

Tous les opérateurs XPath ne sont pas pris en charge. Consultez [la documentation](https://docs.python.org/3/library/xml.etree.elementtree.html#xpath-support) pour une référence complète.

## Modifier un fichier XML

L’écriture de fichiers XML s’effectue élément par élément grâce aux classes `Element` et `SubElement` :

In [ ]:
import xml.etree.ElementTree as ET

sentence = ['Le', 'petit', 'chat', 'est', 'mort', '.']
sent = ET.Element('sent')

for word in sentence:
    w = ET.SubElement(sent, 'w')
    w.text = word

Pour afficher la sortie, utiliser la méthode `.dump()` :

In [ ]:
ET.dump(sent)

La méthode `.set()` permet d’enregistrer un attribut :

In [ ]:
import xml.etree.ElementTree as ET

sentence = {
    'Le': 'DET', 'petit': 'ADJ', 'chat': 'NC',
    'est': 'V', 'mort': 'ADJ', '.': 'PONCT'
}

sent = ET.Element('sent')

for word, pos in sentence.items():
    w = ET.SubElement(sent, 'w')
    w.text = word
    w.set('pos', pos)

La méthode `.write()`, enfin, permet d’écrire le fichier XML à proprement parler :

In [ ]:
tree = ET.ElementTree(sent)
tree.write('./data/the-little-cat.xml')